In [ ]:
from pathlib import Path
import numpy as np
import cv2
import matplotlib.pyplot as plt
ROOT = Path("..")  # چون notebook داخل notebooks/ است
IMG_DIR = ROOT / "data" / "raw" / "images"
MSK_DIR = ROOT / "data" / "raw" / "masks"

images = sorted(IMG_DIR.glob("*.png"))
masks  = sorted(MSK_DIR.glob("*.png"))

len(images), len(masks)
image_names = {p.name for p in images}
mask_names  = {p.name for p in masks}

missing_masks = sorted(list(image_names - mask_names))  # باید 5 تا باشه: 025-029
extra_masks   = sorted(list(mask_names - image_names))  # باید خالی باشه

missing_masks[:10], extra_masks


“Missing masks are expected for test images.”

In [ ]:
def load_rgb(p):
    img = cv2.imread(str(p))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

def load_mask(p):
    m = cv2.imread(str(p), cv2.IMREAD_GRAYSCALE)
    return m

def show_sample(name):
    img = load_rgb(IMG_DIR / name)
    msk = load_mask(MSK_DIR / name)

    msk01 = (msk > 127).astype(np.uint8)

    overlay = img.copy()
    overlay[msk01 == 1, 0] = np.clip(overlay[msk01 == 1, 0] + 80, 0, 255)

    plt.figure(figsize=(12,4))
    plt.subplot(1,3,1); plt.title("Image"); plt.imshow(img); plt.axis("off")
    plt.subplot(1,3,2); plt.title("Mask"); plt.imshow(msk, cmap="gray"); plt.axis("off")
    plt.subplot(1,3,3); plt.title("Overlay"); plt.imshow(overlay); plt.axis("off")
    plt.show()
for n in ["000.png", "005.png", "012.png", "024.png"]:
    show_sample(n)
unique_vals = set()
for p in masks:
    m = load_mask(p)
    unique_vals.update(np.unique(m).tolist())

sorted(list(unique_vals))[:20], len(unique_vals)
coverages = []
for p in masks:
    m = load_mask(p)
    m01 = (m > 127).astype(np.float32)
    coverages.append(m01.mean())

print("Min/Mean/Max roof coverage:", np.min(coverages), np.mean(coverages), np.max(coverages))
plt.hist(coverages, bins=10)
plt.xlabel("Roof pixel ratio"); plt.ylabel("count")
plt.show()
shapes = []
for p in images:
    img = cv2.imread(str(p))
    shapes.append(img.shape[:2])

set(shapes)
